In [2]:
%matplotlib inline

import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import scale as mscale
from matplotlib import transforms as mtransforms
import matplotlib.cm as cm

from matplotlib import pyplot as plt
from scipy.stats import binom
from utils import *

In [22]:
# Grab the individual strain data
poolcount1_fname = "../data/2017_11_03_miseq_barSeq1/all.poolcount"
poolcount2_fname = "../data/2017_11_03_miseq_barSeq2/all.poolcount"

t0_label = 'test.S1'
hc_label = 'test.S2'  # high CO2
lc_label = 'test.S3'  # low CO2 = ambient

poolcount1_df = pd.read_csv(poolcount1_fname, sep='\t')
poolcount2_df = pd.read_csv(poolcount2_fname, sep='\t')

# Merge data from the two replicates
merge_on = ["barcode", "rcbarcode", "scaffold", "strand", "pos", "locusId"]
poolcount_df = poolcount1_df.merge(poolcount2_df, how='outer', on=merge_on)
cols2drop = set(poolcount_df.columns) - set(merge_on)
poolcount_df.drop(cols2drop, axis=1, inplace=True)
poolcount_df

,barcode,rcbarcode,scaffold,strand,pos,locusId
0,GTAGTATTTCAGTTGTGGTG,CACCACAACTGAAATACTAC,NC_013422.1,-,30.0,NaN
1,AGATCGACATGATAAGGGGG,CCCCCTTATCATGTCGATCT,NC_013422.1,-,40.0,NaN
2,TAGTATTGTTGCAGGGTGTT,AACACCCTGCAACAATACTA,NC_013422.1,-,62.0,NaN
3,AAAGCCCGATAGCCACTGAC,GTCAGTGGCTATCGGGCTTT,NC_013422.1,+,71.0,NaN
4,CTATCGGGGCGCGCGGCCGT,ACGGCCGCGCGCCCCGATAG,NC_013422.1,+,103.0,NaN
5,TCACGCGGAGGGTAGGGAAT,ATTCCCTACCCTCCGCGTGA,NC_013422.1,+,104.0,NaN
6,ATAATATTGGACGCGCGGTG,CACCGCGCGTCCAATATTAT,NC_013422.1,-,105.0,NaN
7,GGTTTGGGGCGGCCCTGAGG,CCTCAGGGCCGCCCCAAACC,NC_013422.1,+,108.0,NaN
8,ACAAAAGGTCTGACGCCGGG,CCCGGCGTCAGACCTTTTGT,NC_013422.1,-,110.0,NaN
9,GGGAGGGCGGGCCCGTGGGG,CCCCACGGGCCCGCCCTCCC,NC_013422.1,+,113.0,NaN


In [24]:
poolcount_df.groupby('locusId').count().barcode

locusId
GFF1190          151
GFF1209           16
GFF1357           67
GFF1439           86
GFF1496            6
GFF160           113
GFF1606           13
GFF1629           23
GFF1646            6
GFF1704           35
GFF1835           50
GFF2046           17
GFF2212           26
GFF2252          114
GFF2279           21
GFF2280           15
GFF2288            4
GFF365            44
GFF366            33
GFF371            16
GFF455            47
GFF511            17
GFF559            10
GFF718           120
GFF782            57
GFF824            16
GFF879            19
GFF880            56
GFF881            28
GFF94             24
                ... 
HNEAP_RS12385     33
HNEAP_RS12390     36
HNEAP_RS12395     12
HNEAP_RS12400     52
HNEAP_RS12405     31
HNEAP_RS12410     13
HNEAP_RS12420     10
HNEAP_RS12425    111
HNEAP_RS12430     11
HNEAP_RS12440      2
HNEAP_RS12445    137
HNEAP_RS12450     24
HNEAP_RS12455    240
HNEAP_RS12465     33
HNEAP_RS12470      6
HNEAP_RS12475      6
HNEAP

In [30]:
pd.DataFrame(data = poolcount_df.groupby('locusId').count().barcode)

,barcode
locusId,
GFF1190,151
GFF1209,16
GFF1357,67
GFF1439,86
GFF1496,6
GFF160,113
GFF1606,13
GFF1629,23
GFF1646,6
